In [47]:
!pip install PyPDF2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [111]:
from sentence_transformers import SentenceTransformer, util
import torch
import PyPDF2
import re

In [113]:
text = [
    "Искусственный интеллект — это область компьютерных наук, которая занимается созданием систем, способных выполнять задачи, требующие человеческого интеллекта.",
    "Машинное обучение — это подраздел искусственного интеллекта, который позволяет компьютерам учиться на данных без явного программирования.",
    "Глубокое обучение использует нейронные сети для обработки больших объемов данных и решения сложных задач.",
    "Обработка естественного языка — это раздел искусственного интеллекта, который занимается взаимодействием между компьютерами и людьми через текст или речь.",
    "Семантический поиск — это метод поиска информации, который учитывает смысл запроса и контекст документов."
]

In [115]:
# Шаг 1: Инициализация модели
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [116]:
# Шаг 2: Преобразование текста в векторы
text_embeddings = model.encode(text, convert_to_tensor=True)

In [117]:
# Шаг 3: Функция для семантического поиска
def semantic_search(query, top_k=3):
    # Преобразуем запрос в вектор
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Вычисляем косинусное сходство между запросом и документами
    cos_scores = util.cos_sim(query_embedding, text_embeddings)[0]
    
    # Сортируем результаты по убыванию сходства
    top_results = torch.topk(cos_scores, k=top_k)
    
    # Возвращаем подходящие результаты
    results = []
    for idx, score in zip(top_results.indices, top_results.values):
        results.append({
            "text": text[idx],
            "score": score.item()
        })
    return results

In [118]:
# Демонстрация работы
query = "Что такое машинное обучение?"
results = semantic_search(query)

print("Результаты поиска:")
# Наиболее подходящий результат первый в выводе
for result in results:
    print(f"Сходство: {result['score']:.4f}")
    print(f"Текст: {result['text']}\n")

Результаты поиска:
Сходство: 0.7884
Текст: Машинное обучение — это подраздел искусственного интеллекта, который позволяет компьютерам учиться на данных без явного программирования.

Сходство: 0.5157
Текст: Искусственный интеллект — это область компьютерных наук, которая занимается созданием систем, способных выполнять задачи, требующие человеческого интеллекта.

Сходство: 0.5148
Текст: Обработка естественного языка — это раздел искусственного интеллекта, который занимается взаимодействием между компьютерами и людьми через текст или речь.



In [125]:
from sentence_transformers import SentenceTransformer, util
import torch
import PyPDF2
import re

# Функция для извлечения текста из PDF
def extract_text_from_pdf(pdf_path):
    """
    Извлекает текст из PDF-файла.
    
    :param pdf_path: Путь к PDF-файлу.
    :return: Строка с текстом из PDF.
    """
    text = ""
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                extracted_page_text = page.extract_text()
                if extracted_page_text:  # Проверяем, что текст извлечён
                    text += extracted_page_text
    except Exception as e:
        print(f"Ошибка при чтении PDF: {e}")
    return text.strip()  # Убираем лишние пробелы и пустые строки


# Функция для разделения текста на предложения
def split_into_sentences(text):
    """
    Разделяет текст на предложения с помощью регулярных выражений.
    
    :param text: Исходный текст.
    :return: Список предложений.
    """
    # Регулярное выражение для разделения текста на предложения
    sentences = re.split(r'(?<=[.!?])\s+', text)
    return [sentence.strip() for sentence in sentences if sentence.strip()]


# Шаг 1: Инициализация модели
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Шаг 2: Загрузка текста из PDF
pdf_path = 'natural1.pdf'  # Укажите путь к вашему PDF-файлу
extracted_text = extract_text_from_pdf(pdf_path)

# Если текст успешно извлечён, используем его как корпус
if extracted_text:
    # Разделяем текст на предложения
    text = split_into_sentences(extracted_text)
    if not text:
        raise ValueError("Извлечённый текст пуст или не содержит предложений.")
else:
    raise ValueError("Не удалось извлечь текст из PDF.")

print(f"Количество предложений в корпусе: {len(text)}")

# Шаг 3: Преобразование корпуса в векторы
text_embeddings = model.encode(text, convert_to_tensor=True)

# Шаг 4: Функция для семантического поиска
def semantic_search(query, top_k=3):
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Вычисляем косинусное сходство между запросом и документами
    cos_scores = util.cos_sim(query_embedding, text_embeddings)[0]
    
    # Ограничиваем top_k до количества доступных документов
    top_k = min(top_k, len(text))
    
    # Сортируем результаты по убыванию сходства
    top_results = torch.topk(cos_scores, k=top_k)
    
    # Возвращаем подходящие результаты
    results = []
    for idx, score in zip(top_results.indices, top_results.values):
        results.append({
            "text": text[idx],
            "score": score.item()
        })
    return results


# Демонстрация работы
query = "Где лежат истоки европейской цивилизации?"
try:
    results = semantic_search(query)
except RuntimeError as e:
    print(f"Ошибка при выполнении поиска: {e}")
else:
    print("Результаты поиска:")
    for result in results:
        print(f"Сходство: {result['score']:.4f}")
        print(f"Текст: {result['text'][:100]}...\n")  # Выводим первые 100 символов текста

Multiple definitions in dictionary at byte 0x93f for key /F19
Multiple definitions in dictionary at byte 0x94c for key /F33
Multiple definitions in dictionary at byte 0x12b43 for key /F19
Multiple definitions in dictionary at byte 0x13817 for key /F19
Multiple definitions in dictionary at byte 0x14570 for key /F19
Multiple definitions in dictionary at byte 0x15312 for key /F19
Multiple definitions in dictionary at byte 0x16087 for key /F19
Multiple definitions in dictionary at byte 0x16df9 for key /F19
Multiple definitions in dictionary at byte 0x17bca for key /F19
Multiple definitions in dictionary at byte 0x18904 for key /F19
Multiple definitions in dictionary at byte 0x1960d for key /F19
Multiple definitions in dictionary at byte 0x1a365 for key /F19
Multiple definitions in dictionary at byte 0x1b06e for key /F19
Multiple definitions in dictionary at byte 0x1be08 for key /F19
Multiple definitions in dictionary at byte 0x1cb8a for key /F19
Multiple definitions in dictionary at byte 0

Количество предложений в корпусе: 524
Результаты поиска:
Сходство: 0.6080
Текст: Истоки европейской циви-
лизации лежат в античной Греции, создавшей культуру, которая
обладает удиви...

Сходство: 0.5399
Текст: Древние греки
явились родоначальниками того рационального гуманизма , кото-
рый и сегодня определяет...

Сходство: 0.5004
Текст: НАУКА И ЕЕ РОЛЬ В СОВРЕМЕННОМ МИРЕ
Цивилизация, культура, наука....



In [129]:
# Вопрос из середины пдф файла
query = "В чём состоит основная задача изучения естествознания?"
try:
    results = semantic_search(query)
except RuntimeError as e:
    print(f"Ошибка при выполнении поиска: {e}")
else:
    print("Результаты поиска:")
    for result in results:
        print(f"Сходство: {result['score']:.4f}")
        print(f"Текст: {result['text'][:100]}...\n")  # Выводим первые 100 символов текста

Результаты поиска:
Сходство: 0.8192
Текст: Основная задача изучения естествознания состоит в том, чтобы
ясно представить единство природы, ее ц...

Сходство: 0.8008
Текст: Природа и ее изучение, естествознание....

Сходство: 0.7444
Текст: Именно в этом смысле естествознание
рассматривается как единая система наук о природе, занимающая-
с...



In [133]:
# Вопрос из конца пдф документа
query = "Возможности науки?"
try:
    results = semantic_search(query)
except RuntimeError as e:
    print(f"Ошибка при выполнении поиска: {e}")
else:
    print("Результаты поиска:")
    for result in results:
        print(f"Сходство: {result['score']:.4f}")
        print(f"Текст: {result['text'][:100]}...\n")  # Выводим первые 100 символов текста

Результаты поиска:
Сходство: 0.7796
Текст: Но наука и не всесильна....

Сходство: 0.7768
Текст: Одной науки для этого
мало, но, чтобы достичь желаемого, вовсе не надо отворачиваться
от науки....

Сходство: 0.7548
Текст: •Возможности науки
Сегодня ученые вовсе не считают, что они постигли все тайны
мироздания....

